
# torchvision.transforms 方法实战笔记

本笔记演示 `torchvision.transforms.transforms` 模块中每个公开方法的用途、对图像的影响以及在项目 `dataset/arch_linux` 图像上的具体效果，便于快速对照与复用。


## 变换类型速览

| 分类 | 代表 API | 功能概述 | 常见用途 |
| --- | --- | --- | --- |
| 几何类 | `RandomResizedCrop`, `ElasticTransform`, `RandomPerspective` | 旋转、缩放、裁剪、弹性形变等，改变空间结构 | 模拟视角/位置变化，增强几何鲁棒性 |
| 颜色/光照 | `ColorJitter`, `RandomAutocontrast`, `RandomSolarize` | 调整亮度、对比度、颜色、曝光 | 对抗不同光照、摄像头色偏 |
| 噪声/遮挡 | `RandomErasing`, `GaussianBlur`, `RandomPosterize` | 引入遮挡、模糊或量化噪声 | 模拟遮挡、压缩损失、失焦 |
| 结构化组合 | `Compose`, `RandomApply`, `RandomChoice`, `RandomOrder` | 将基础变换按顺序或概率组合 | 构建可复用增广流水线，控制执行逻辑 |
| 数据类型 | `ToTensor`, `PILToTensor`, `ConvertImageDtype`, `Normalize` | 模式/数据类型转换与归一化 | 构建符合模型输入的数据、支持混合精度 |
| TVTensor 生态 | `v2.ToImage`, `tv_tensors.Image` 等 | 携带尺寸/坐标元数据的张量类型 | 同步处理图像、框、关键点、掩码 |
| 自定义扩展 | `Lambda`, `RandomTransforms` | 注入定制逻辑或随机策略 | 针对特殊需求快速试验 |
| 多视角裁剪 | `FiveCrop`, `TenCrop` | 一次生成多个裁剪视角 | 测试时平均，提升稳定性 |

In [ ]:

from pathlib import Path
from PIL import Image
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import torchvision.transforms as T
import torchvision.transforms.transforms as TT

os.environ.setdefault('OMP_NUM_THREADS', '1')
torch.set_num_threads(1)

DATASET_ROOT = Path('dataset/arch_linux')
SAMPLE_IMAGE = sorted(DATASET_ROOT.glob('*.png'))[0]
base_pil = Image.open(SAMPLE_IMAGE).convert('RGB')
base_tensor = T.ToTensor()(base_pil)

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['figure.facecolor'] = 'white'

# 辅助函数：获取基础 PIL 图像和张量副本
def get_base_pil():
    return base_pil.copy()

# 辅助函数：获取基础张量副本
def get_base_tensor():
    return base_tensor.clone()

# 辅助函数：重置随机种子
def reset_seed(seed: int = 42):
    torch.manual_seed(seed)

# 辅助函数：将张量或 PIL 图像转换为可显示的 PIL 图像
def to_displayable(img):
    if isinstance(img, Image.Image):
        return img
    if isinstance(img, torch.Tensor):
        tensor = img.detach().clone()
        if tensor.dtype not in (torch.float32, torch.float64):
            tensor = tensor.float()
        if tensor.ndim == 3 and tensor.shape[0] in (1, 3):
            tensor = tensor.clamp(0, 1)
            return T.ToPILImage()(tensor)
        raise TypeError(f'Unsupported tensor shape: {tensor.shape}')
    raise TypeError(f'Unsupported type: {type(img)}')

# 辅助函数：显示图像网格, 支持 PIL 图像和张量
def show_images(images, titles=None, cols=2, figsize=(8, 4)):
    pil_images = [to_displayable(img) for img in images]
    total = len(pil_images)
    if total == 0:
        return
    cols = min(cols, total)
    rows = (total + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    if isinstance(axes, np.ndarray):
        axes = axes.reshape(-1)
    else:
        axes = np.array([axes], dtype=object)
    titles = list(titles or [])
    if len(titles) < total:
        titles.extend([''] * (total - len(titles)))
    for idx, img in enumerate(pil_images):
        ax = axes[idx]
        ax.imshow(img)
        ax.axis('off')
        if titles[idx]:
            ax.set_title(titles[idx])
    for ax in axes[len(pil_images):]:
        ax.axis('off')
    plt.tight_layout()



### CenterCrop 中心裁剪
- **目的**：从图像中心裁剪指定尺寸，常用于推理阶段去除边缘噪声。
- **对图片的作用**：保留中心区域，剔除四周背景，让主体更突出。


In [ ]:


reset_seed()
transform = T.CenterCrop((512, 512))
original = get_base_pil()
result = transform(original)
show_images([original, result], ["原图", "CenterCrop 后"], cols=2)



### ColorJitter 颜色抖动
- **目的**：随机扰动亮度、对比度、饱和度和色相，模拟不同拍摄光照。
- **对图片的作用**：输出颜色变化的版本，让模型对光照和颜色更鲁棒。


In [ ]:

reset_seed()
transform = T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05)
original = get_base_pil()
result = transform(original)
show_images([original, result], ["原图", "ColorJitter 后"], cols=2)



### Compose 复合组合
- **目的**：顺序组合多个基本变换，构建可复用的数据增强流水线。
- **对图片的作用**：依次调整尺寸、翻转与颜色，得到规范化的增强图像。


In [ ]:

reset_seed()
pipeline = T.Compose([
    T.Resize((512, 512)),
    T.RandomHorizontalFlip(p=1.0),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),
])
original = get_base_pil()
result = pipeline(original)
show_images([original, result], ["原图", "Compose 流水线结果"], cols=2)



### ConvertImageDtype 图像数据类型转换
- **目的**：将张量图像转换为指定 dtype 并自动缩放数值范围。
- **对图片的作用**：示例中把 float32 转为 float16，便于混合精度训练。


In [ ]:

import torch
reset_seed()
tensor = get_base_tensor()
transform = TT.ConvertImageDtype(torch.float16)
converted = transform(tensor)
print(f"原始 dtype: {tensor.dtype}, 转换后: {converted.dtype}")
show_images([tensor, converted.float()], ["原始张量", "float16 张量"], cols=2)



### ElasticTransform 弹性形变
- **目的**：对张量图像施加弹性位移，模拟柔性扭曲场景。
- **对图片的作用**：局部区域发生非线性形变，逼近摄像抖动或组织形变。


In [ ]:

reset_seed()
transform = T.ElasticTransform(alpha=50.0, sigma=5.0)
source = get_base_tensor()
warped = transform(source)
show_images([source, warped], ["原始张量", "ElasticTransform 后"], cols=2)



### FiveCrop 五点裁剪
- **目的**：返回左上、右上、左下、右下以及中心裁剪，共五个视角。
- **对图片的作用**：一次性生成多个裁剪，用于测试时平均预测提高稳定性。


In [ ]:

reset_seed()
transform = T.FiveCrop(size=256)
original = get_base_pil()
crops = transform(original)
show_images([original] + list(crops), ["原图"] + [f"裁剪 {i+1}" for i in range(len(crops))], cols=3, figsize=(12, 8))



### GaussianBlur 高斯模糊
- **目的**：以随机 sigma 执行高斯模糊，模拟失焦或低清成像。
- **对图片的作用**：输出图像更平滑，细节被软化，有助于提升鲁棒性。


In [ ]:

reset_seed()
transform = T.GaussianBlur(kernel_size=9, sigma=(0.5, 1.5))
original = get_base_pil()
blurred = transform(original)
show_images([original, blurred], ["原图", "GaussianBlur 后"], cols=2)



### Grayscale 灰度化
- **目的**：将图像转换为灰度，可选择输出 1 或 3 个通道。
- **对图片的作用**：示例保留 3 通道灰度图，强调亮度纹理。


In [ ]:

reset_seed()
transform = T.Grayscale(num_output_channels=3)
original = get_base_pil()
gray = transform(original)
show_images([original, gray], ["原图", "灰度图"], cols=2)



### Lambda 自定义 Lambda 变换
- **目的**：支持自定义 Python 函数插入到变换流水线。
- **对图片的作用**：示例将像素截断到 0.8 以下，抑制过亮区域。


In [ ]:

reset_seed()
transform = TT.Lambda(lambda tensor: tensor.clamp(0.0, 0.8))
source = get_base_tensor()
processed = transform(source)
show_images([source, processed], ["原始张量", "截断亮度"], cols=2)



### LinearTransformation 线性变换
- **目的**：对展平后的张量执行线性变换，可实现白化或投影。
- **对图片的作用**：示例对 32×32 缩小图像减去常量均值，突出暗部细节。


In [ ]:

import torch
reset_seed()
resize_small = T.Resize((32, 32))
small_pil = resize_small(get_base_pil())
small_tensor = T.ToTensor()(small_pil)
numel = small_tensor.numel()
matrix = torch.eye(numel)
mean_vector = torch.full((numel,), 0.4)
transform = TT.LinearTransformation(matrix, mean_vector)
transformed = transform(small_tensor)
visual = transformed.clone()
visual = (visual - visual.min()) / (visual.max() - visual.min() + 1e-6)
show_images([small_pil, T.ToPILImage()(visual)], ["缩小原图", "线性变换后"], cols=2)



### Normalize 标准化
- **目的**：按照给定均值与方差对张量执行按通道标准化。
- **对图片的作用**：输出符合预训练模型分布，同时打印新统计量供检查。


In [ ]:

import torch
reset_seed() # 重置随机种子，确保结果可重复
source = get_base_tensor() # 原始张量
normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # 归一化变换
normalized = normalize(source.clone()) # 归一化后的张量
print("归一化前均值:", source.mean(dim=(1, 2))) # 打印原始张量的通道均值
print("归一化后均值:", normalized.mean(dim=(1, 2))) # 打印归一化张量的通道均值
visual = normalized.clone() # 可视化张量，需要将归一化后的张量转换为 PIL 图像

# 归一化到 0-1 范围，避免显示问题
visual = (visual - visual.min()) / (visual.max() - visual.min() + 1e-6)

# 可视化归一化后的张量
show_images([source, visual], ["原始张量", "Normalize 后"], cols=2)



### PILToTensor PIL 转张量
- **目的**：将 PIL 图像转换为保持 0-255 范围的 uint8 张量。
- **对图片的作用**：示例展示形状与 dtype，便于后续自定义张量处理。


In [ ]:

tensor = T.PILToTensor()(get_base_pil())
print(f"输出 dtype: {tensor.dtype}, 形状: {tuple(tensor.shape)}")
show_images([tensor.float() / 255.0], ["PILToTensor 结果"])



### Pad 填充
- **目的**：在图像四周补边，可用于裁剪前保留更多背景。
- **对图片的作用**：示例使用常量填充 40 像素，扩展画布大小。


In [ ]:

reset_seed()
transform = T.Pad(padding=40, fill=30, padding_mode="constant")
original = get_base_pil()
padded = transform(original)
show_images([original, padded], ["原图", "Pad 后"], cols=2)



### RandomAdjustSharpness 随机锐度调整
- **目的**：随机调整锐度，强化或柔化边缘细节。
- **对图片的作用**：示例将锐度提升 1.5 倍，增强图像边缘。


In [ ]:

reset_seed()
transform = T.RandomAdjustSharpness(sharpness_factor=1.5, p=1.0)
original = get_base_pil()
sharpened = transform(original)
show_images([original, sharpened], ["原图", "锐度增强"], cols=2)



### RandomAffine 随机仿射变换
- **目的**：随机执行旋转、平移、缩放与错切的组合仿射变换。
- **对图片的作用**：示例中包含 15° 旋转与 10% 平移，模拟视角变化。


In [ ]:

reset_seed()
transform = T.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10)
original = get_base_pil()
warped = transform(original)
show_images([original, warped], ["原图", "RandomAffine 后"], cols=2)



### RandomApply 随机应用组合
- **目的**：以给定概率整体执行一组变换。
- **对图片的作用**：示例设定 p=1.0，确保颜色扰动与翻转全部生效。


In [ ]:

reset_seed()
transform = T.RandomApply([
    T.ColorJitter(0.2, 0.2, 0.2, 0.05),
    T.RandomHorizontalFlip(p=1.0),
], p=1.0)
original = get_base_pil()
augmented = transform(original)
show_images([original, augmented], ["原图", "RandomApply 后"], cols=2)



### RandomAutocontrast 随机自动对比度
- **目的**：随机执行自动对比度拉伸，扩展像素动态范围。
- **对图片的作用**：示例中强制处理一次，暗部更亮、亮部更突出。


In [ ]:

reset_seed()
transform = T.RandomAutocontrast(p=1.0)
original = get_base_pil()
processed = transform(original)
show_images([original, processed], ["原图", "自动对比度"], cols=2)



### RandomChoice 随机选择变换
- **目的**：在给定列表中随机选择一个变换执行。
- **对图片的作用**：示例候选包括颜色扰动、水平翻转和灰度化，种子固定以重现结果。


In [ ]:

reset_seed()
transform = TT.RandomChoice([
    T.ColorJitter(0.3, 0.3, 0.3, 0.05),
    T.RandomHorizontalFlip(p=1.0),
    T.RandomGrayscale(p=1.0),
])
original = get_base_pil()
result = transform(original)
show_images([original, result], ["原图", "RandomChoice 结果"], cols=2)



### RandomCrop 随机裁剪
- **目的**：随机位置裁剪，常与填充配合提升背景多样性。
- **对图片的作用**：示例裁剪 400×400 区域，聚焦于不同局部。


In [ ]:

reset_seed()
transform = T.RandomCrop(size=(400, 400))
original = get_base_pil()
cropped = transform(original)
show_images([original, cropped], ["原图", "随机裁剪"], cols=2)



### RandomEqualize 随机直方图均衡
- **目的**：随机对图像执行直方图均衡化。
- **对图片的作用**：示例强制均衡直方图，提升暗部对比度。


In [ ]:

reset_seed()
transform = T.RandomEqualize(p=1.0)
original = get_base_pil()
processed = transform(original)
show_images([original, processed], ["原图", "直方图均衡化"], cols=2)



### RandomErasing 随机擦除
- **目的**：在张量图像上随机擦除矩形区域，模拟遮挡。
- **对图片的作用**：示例以 100% 概率擦除面积占比 20%-30% 的区域。


In [ ]:

import torch
reset_seed()
transform = T.RandomErasing(p=1.0, scale=(0.2, 0.3), ratio=(0.3, 3.3), value=0.5)
source = get_base_tensor()
occluded = transform(source.clone())
show_images([source, occluded], ["原始张量", "RandomErasing 后"], cols=2)



### RandomGrayscale 随机灰度化
- **目的**：以概率将图像转换为灰度，同时可保持 3 通道输出。
- **对图片的作用**：示例强制转灰度，帮助模型关注纹理。


In [ ]:

reset_seed()
transform = T.RandomGrayscale(p=1.0)
original = get_base_pil()
processed = transform(original)
show_images([original, processed], ["原图", "随机灰度"], cols=2)



### RandomHorizontalFlip 随机水平翻转
- **目的**：以概率执行水平翻转。
- **对图片的作用**：示例始终左右镜像，模拟视角对称。


In [ ]:

reset_seed()
transform = T.RandomHorizontalFlip(p=1.0)
original = get_base_pil()
flipped = transform(original)
show_images([original, flipped], ["原图", "水平翻转"], cols=2)



### RandomInvert 随机反色
- **目的**：以概率对图像执行颜色反转。
- **对图片的作用**：示例生成负片风格输出，提高对反色场景的适应力。


In [ ]:

reset_seed()
transform = T.RandomInvert(p=1.0)
original = get_base_pil()
inverted = transform(original)
show_images([original, inverted], ["原图", "颜色反转"], cols=2)



### RandomOrder 随机顺序变换
- **目的**：随机重排一组变换的执行顺序。
- **对图片的作用**：示例在色彩与翻转之间打乱顺序，生成多样化结果。


In [ ]:

reset_seed()
transform = TT.RandomOrder([
    T.ColorJitter(0.3, 0.3, 0.3, 0.05),
    T.RandomHorizontalFlip(p=1.0),
])
original = get_base_pil()
result = transform(original)
show_images([original, result], ["原图", "RandomOrder 后"], cols=2)



### RandomPerspective 随机透视变换
- **目的**：随机执行透视畸变，模拟视角变化。
- **对图片的作用**：示例设置较大的 distortion_scale，突出透视拉伸。


In [ ]:

reset_seed()
transform = T.RandomPerspective(distortion_scale=0.6, p=1.0)
original = get_base_pil()
warped = transform(original)
show_images([original, warped], ["原图", "透视畸变"], cols=2)



### RandomPosterize 随机色彩量化
- **目的**：随机减少位深，模拟低质量量化。
- **对图片的作用**：示例保留 4 bit 色彩，产生分层效果。


In [ ]:

reset_seed()
transform = T.RandomPosterize(bits=4, p=1.0)
original = get_base_pil()
posterized = transform(original)
show_images([original, posterized], ["原图", "4 bit Posterize"], cols=2)



### RandomResizedCrop 随机缩放裁剪
- **目的**：随机裁剪不同面积与宽高比后再缩放到目标尺寸。
- **对图片的作用**：示例裁剪到 512×512，涵盖多尺度信息。


In [ ]:

reset_seed()
transform = T.RandomResizedCrop(size=(512, 512), scale=(0.5, 1.0), ratio=(0.75, 1.33))
original = get_base_pil()
result = transform(original)
show_images([original, result], ["原图", "RandomResizedCrop 后"], cols=2)



### RandomRotation 随机旋转
- **目的**：在给定角度范围内随机旋转图像。
- **对图片的作用**：示例 ±30° 随机旋转，模拟拍摄倾斜。


In [ ]:

reset_seed()
transform = T.RandomRotation(degrees=(-30, 30), expand=True)
original = get_base_pil()
rotated = transform(original)
show_images([original, rotated], ["原图", "随机旋转"], cols=2)



### RandomSolarize 随机日晒效果
- **目的**：随机对高于阈值的像素执行反转，模拟过曝。
- **对图片的作用**：示例阈值 100，亮部被翻转产生冲击感。


In [ ]:

reset_seed()
transform = T.RandomSolarize(threshold=100, p=1.0)
original = get_base_pil()
processed = transform(original)
show_images([original, processed], ["原图", "Solarize"], cols=2)



### RandomTransforms 随机变换基类
- **目的**：自定义带随机性的复合变换基类，需要实现 __call__。
- **对图片的作用**：示例随机选择水平翻转或旋转，展示自定义用法。


In [ ]:

import torch
class FlipOrRotate(TT.RandomTransforms):
    def __init__(self):
        super().__init__([
            T.RandomHorizontalFlip(p=1.0),
            T.RandomRotation(20),
        ])

    def __call__(self, img):
        idx = torch.randint(len(self.transforms), (1,)).item()
        return self.transforms[idx](img)

reset_seed()
transform = FlipOrRotate()
original = get_base_pil()
augmented = transform(original)
show_images([original, augmented], ["原图", "自定义 RandomTransforms"], cols=2)



### RandomVerticalFlip 随机垂直翻转
- **目的**：以概率执行垂直翻转。
- **对图片的作用**：示例始终上下翻转，模拟镜像倒置场景。


In [ ]:

reset_seed()
transform = T.RandomVerticalFlip(p=1.0)
original = get_base_pil()
flipped = transform(original)
show_images([original, flipped], ["原图", "垂直翻转"], cols=2)



### Resize 缩放
- **目的**：调整图像尺寸，可按长边或指定大小缩放。
- **对图片的作用**：示例统一缩放到 512×512，便于批处理对齐。


In [ ]:

transform = T.Resize((512, 512))
original = get_base_pil()
resized = transform(original)
show_images([original, resized], ["原图", "Resize 后"], cols=2)



### TenCrop 十点裁剪
- **目的**：输出五个裁剪及其垂直翻转，一共十张图像。
- **对图片的作用**：提供多视角输入，常在评估时平均预测。


In [ ]:

reset_seed()
transform = T.TenCrop(size=256, vertical_flip=True)
original = get_base_pil()
patches = transform(original)
show_images(list(patches), [f"裁剪 {i+1}" for i in range(len(patches))], cols=5, figsize=(14, 6))



### ToPILImage 张量转 PIL 图像
- **目的**：将张量或 ndarray 转回 PIL 图像，便于保存与可视化。
- **对图片的作用**：示例把张量恢复为 PIL，再次用于展示。


In [ ]:

tensor = get_base_tensor()
pil_img = T.ToPILImage()(tensor)
show_images([tensor, pil_img], ["张量", "转换为 PIL"], cols=2)



### ToTensor 图像转张量
- **目的**：将 PIL 图像或 ndarray 转换为 float32 张量并归一化到 [0,1]。
- **对图片的作用**：示例输出形状与 dtype，展示数值范围。


In [ ]:

tensor = T.ToTensor()(get_base_pil())
print(f"输出 dtype: {tensor.dtype}, 最小值: {tensor.min():.3f}, 最大值: {tensor.max():.3f}")
show_images([tensor], ["ToTensor 结果"])



### _check_sequence_input 序列参数校验
- **目的**：内部校验工具，确保序列参数满足长度要求。
- **对图片的作用**：示例展示合法输入通过与非法输入抛出异常的差别。


In [ ]:

from torchvision.transforms.transforms import _check_sequence_input
_check_sequence_input((224, 224), "size", (2,))
print("合法输入通过校验。")
try:
    _check_sequence_input((224,), "size", (2,))
except ValueError as err:
    print(f"非法输入触发异常: {err}")



### _setup_angle 角度参数设置
- **目的**：将角度参数规范化为 (min, max) 形式，供仿射等变换使用。
- **对图片的作用**：示例说明单值与区间输入都被转换为统一区间。


In [ ]:

from torchvision.transforms.transforms import _setup_angle
print("单值输入:", _setup_angle(15, "degrees"))
print("区间输入:", _setup_angle((-30, 30), "degrees"))



### _setup_size 尺寸参数解析
- **目的**：解析 size 参数为 (height, width) 并做合法性检查。
- **对图片的作用**：示例展示多种输入格式的解析结果。


In [ ]:

from torchvision.transforms.transforms import _setup_size
print("整数输入:", _setup_size(256, "size"))
print("元组输入:", _setup_size((256, 320), "size"))
